In [1]:
import pandas as pd

In [2]:
train=pd.read_csv('train.csv')
train

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation
...,...,...,...
31957,31958,0,ate @user isz that youuu?ðððððð...
31958,31959,0,to see nina turner on the airwaves trying to...
31959,31960,0,listening to sad songs on a monday morning otw...
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,..."


In [3]:
test=pd.read_csv('test.csv')
test

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."
...,...,...
17192,49155,thought factory: left-right polarisation! #tru...
17193,49156,feeling like a mermaid ð #hairflip #neverre...
17194,49157,#hillary #campaigned today in #ohio((omg)) &am...
17195,49158,"happy, at work conference: right mindset leads..."


# DATA CLEANING AND PREPROCESSING

In [10]:
#To remove twitter usernames,special characters,URLs,http etc..from the dataframe

import re

def cleaned_text(df,text_field):
    df[text_field]=df[text_field].str.lower()
    df[text_field]=df[text_field].apply(lambda x:re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?","",x))
    return df

train_cleaned=cleaned_text(train,'tweet')
test_cleaned=cleaned_text(test,'tweet')

In [11]:
train_cleaned

,id,label,tweet
0,1,0,when a father is dysfunctional and is so sel...
1,2,0,thanks for lyft credit i cant use cause they...
2,3,0,bihday your majesty
3,4,0,model i love u take with u all the time in u...
4,5,0,factsguide society now motivation
...,...,...,...
31957,31958,0,ate isz that youuu
31958,31959,0,to see nina turner on the airwaves trying to...
31959,31960,0,listening to sad songs on a monday morning otw...
31960,31961,1,sikh temple vandalised in in calgary wso cond...


In [12]:
test_cleaned

,id,tweet
0,31963,studiolife aislife requires passion dedication...
1,31964,white supremacists want everyone to see the ...
2,31965,safe ways to heal your acne altwaystoheal h...
3,31966,is the hp and the cursed child book up for res...
4,31967,3rd bihday to my amazing hilarious nephew el...
...,...,...
17192,49155,thought factory leftright polarisation trump u...
17193,49156,feeling like a mermaid hairflip neverready fo...
17194,49157,hillary campaigned today in ohioomg amp used w...
17195,49158,happy at work conference right mindset leads t...


In [18]:
train['label'].value_counts()

label
0    29720
1     2242
Name: count, dtype: int64

We observe imbalanced data...count(0)>>>count(1)
Hence we need to handle it.
Here we use oversampling.


In [21]:
from sklearn.utils import resample

train_major=train_cleaned[train_cleaned.label==0]
train_minor=train_cleaned[train_cleaned.label==1]

train_minor_unsampled=resample(train_minor,
                              replace=True,
                              n_samples=len(train_major),
                              random_state=321)

train_sampled=pd.concat([train_minor_unsampled,train_major])
train_sampled

,id,label,tweet
1717,1718,1,you might be a libtard if libtard sjw libera...
26108,26109,1,are you black amp feel like the are stomping ...
20575,20576,1,you all look like 6yearolds posting dictionary...
2475,2476,1,if you are hispanic black asian a nonracist w...
14049,14050,1,spoty perhaps the murrayhaters would be happi...
...,...,...,...
31956,31957,0,off fishing tomorrow carnt wait first time in...
31957,31958,0,ate isz that youuu
31958,31959,0,to see nina turner on the airwaves trying to...
31959,31960,0,listening to sad songs on a monday morning otw...


In [22]:
train_sampled['label'].value_counts()

label
1    29720
0    29720
Name: count, dtype: int64

# PIPELINE CREATION

In [23]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier

pipe=Pipeline([
    ('vector',CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('nb',SGDClassifier())
])

# MODEL TRAINING

In [24]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(train_sampled['tweet'],
                                              train_sampled['label'],
                                              random_state=0)

In [26]:
model=pipe.fit(x_train,y_train)
y_pred=model.predict(x_test)

In [27]:
y_pred

array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

# MODEL EVALUATION

In [29]:
from sklearn.metrics import classification_report

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.98      0.95      0.97      7490
           1       0.95      0.98      0.97      7370

    accuracy                           0.97     14860
   macro avg       0.97      0.97      0.97     14860
weighted avg       0.97      0.97      0.97     14860



                The evaluation metrics is appreciatable